In [66]:
import os
import re

In [67]:
os.listdir()

['.ipynb_checkpoints',
 'data_preprocess.ipynb',
 'mibig_gbk_3.1',
 'mibig_gbk_3.1.tar.gz',
 'mibig_json_3.1',
 'mibig_json_3.1.tar.gz',
 'mibig_prot_seqs_3.1.fasta',
 'output']

In [68]:
files = os.listdir('mibig_gbk_3.1')
print(files)

['BGC0000001.gbk', 'BGC0000002.gbk', 'BGC0000003.gbk', 'BGC0000004.gbk', 'BGC0000006.gbk', 'BGC0000007.gbk', 'BGC0000008.gbk', 'BGC0000009.gbk', 'BGC0000010.gbk', 'BGC0000011.gbk', 'BGC0000012.gbk', 'BGC0000013.gbk', 'BGC0000014.gbk', 'BGC0000016.gbk', 'BGC0000017.gbk', 'BGC0000018.gbk', 'BGC0000019.gbk', 'BGC0000020.gbk', 'BGC0000021.gbk', 'BGC0000022.gbk', 'BGC0000023.gbk', 'BGC0000024.gbk', 'BGC0000025.gbk', 'BGC0000026.gbk', 'BGC0000027.gbk', 'BGC0000028.gbk', 'BGC0000029.gbk', 'BGC0000030.gbk', 'BGC0000031.gbk', 'BGC0000032.gbk', 'BGC0000033.gbk', 'BGC0000034.gbk', 'BGC0000035.gbk', 'BGC0000036.gbk', 'BGC0000037.gbk', 'BGC0000038.gbk', 'BGC0000039.gbk', 'BGC0000040.gbk', 'BGC0000041.gbk', 'BGC0000042.gbk', 'BGC0000043.gbk', 'BGC0000044.gbk', 'BGC0000045.gbk', 'BGC0000046.gbk', 'BGC0000047.gbk', 'BGC0000048.gbk', 'BGC0000050.gbk', 'BGC0000051.gbk', 'BGC0000052.gbk', 'BGC0000053.gbk', 'BGC0000054.gbk', 'BGC0000055.gbk', 'BGC0000056.gbk', 'BGC0000057.gbk', 'BGC0000058.gbk', 'BGC00000

In [69]:
# 使用单个文件进行测试DNA序列
with open('mibig_gbk_3.1/'+'BGC0000001.gbk', 'r') as f:
    text = f.readlines()
#     print(f.readlines())
#     print(text)
#     print(type(text))
    text = ''.join(text)
    DNA_len = int(re.search(r'[0-9]+ bp', text, flags = 0).group()[:-2])
    pattern = re.compile(r'ORIGIN[\s\S]+')
    DNA_seq_num = re.search(pattern, text, flags = 0).group()[7:]
    DNA_seq = re.sub(r'[0-9 \n/]*', '', DNA_seq_num)
    if(DNA_len == len(DNA_seq)):
        print("OK!")
#     print(DNA_seq)    
    print(DNA_seq[58181:58243])

OK!
gcgagttgaaccgccagccgtccgacgcctccggacgcgccggtgaccagcacccgacggcc


In [78]:
# 使用单个文件进行测试，切取单个基因及对应氨基酸序列
reverse_dict = {'a':'t', 't':'a', 'g':'c', 'c':'g'}
with open('mibig_gbk_3.1/'+'BGC0000001.gbk', 'r') as f:
    text = f.readlines() # list格式
    text = ''.join(text) # string格式
    
    DNA_len = int(re.search(r'[0-9]+ bp', text, flags = 0).group()[:-2])
    pattern = re.compile(r'ORIGIN[\s\S]+')
    DNA_seq_num = re.search(pattern, text, flags = 0).group()[7:]
    DNA_seq = re.sub(r'[0-9 \n/]*', '', DNA_seq_num)
    if(DNA_len == len(DNA_seq)):
        print("DNA序列读取数目正确")
    else:
        print("DNA序列读取数目错误")
#         exit(1)
    gene_pattern = re.compile(r'gene            ')
    # gene的匹配需要后面的空格以指示每块的开头，基因互补链也是同样的开头所以可以识别
    gene_set_start = [substr.start() for substr in re.finditer(gene_pattern, text)] + [text.find('ORIGIN')]
    # 因为每个基因可以由多个编码序列（CDS）组成
    if len(gene_set_start)<=1:
        gene_set_set = [1, text.find('ORIGIN')]
    count = 0
    for i in range(len(gene_set_start)-1):
        gene_block = text[gene_set_start[i]:gene_set_start[i+1]]
        CDS_pattern = re.compile(r'CDS             ')
        # 识别CDS，同样可识别complement
        CDS_set_start = [substr.start() for substr in re.finditer(CDS_pattern, gene_block)] + [gene_set_start[i+1]] 
#         print(CDS_set_start)
#         print(gene_block)
        # 存在多个CDS的情况
        for j in range(int(len(CDS_set_start))-1):
            count += 1
            CDS_block = gene_block[CDS_set_start[j]:CDS_set_start[j+1]]
            # 识别正义链的起始
            begin2end_pattern_1 = re.compile(r'CDS             ([0-9]+)..([0-9]+)')
            # 识别反义链的起始
            begin2end_pattern_2 = re.compile(r'CDS             complement\(([0-9]+)..([0-9]+)\)')
            reverse = False
            begin2end_str = re.search(begin2end_pattern_1, CDS_block, flags=0)
            if not begin2end_str:
                begin2end_str = re.search(begin2end_pattern_2, CDS_block, flags=0)
                reverse = True
            translation_pattren = re.compile(r'/translation="([A-Z\s]+)"')
            translation = re.search(translation_pattren, CDS_block, flags=0).group(1)
            translation = translation.replace(' ', '').replace('\n', '')
            startN = int(begin2end_str.group(1))-1 # 索引从0开始
            endN = int(begin2end_str.group(2)) # 包含最后字符
            gene_seq = DNA_seq[startN:endN]
            if reverse:
                gene_seq = ''.join([reverse_dict[ch] for ch in gene_seq])[::-1]
            if len(gene_seq)%3==0:
                print("基因编码长度合理")
            else:
                print("基因编码长度不合理")
#                 break
            print(gene_seq)
            print(translation)
            print(CDS_block)
            print(begin2end_str)
            gene_filename = ('output/'+'BGC0000001.gbk').replace('.gbk', '.gene')
            amino_acid_filename = ('output/'+'BGC0000001.gbk').replace('.gbk', '.aa')
            with open(gene_filename, 'a') as rf:
                rf.write('BGC0000001.gbk'.replace('.gbk', '')+'_gene_'+str(count)+':')
                rf.write(gene_seq+'\n')
            with open(amino_acid_filename, 'a') as rf:
                rf.write('BGC0000001.gbk'.replace('.gbk', '')+'_protein_'+str(count)+':')
                rf.write(translation+'\n')
            
#             print(begin2end_str.group(1), begin2end_str.group(2))
#     print(gene_set_start)
#     print(text[1951:4402])
#     print(text[4402:5319])

DNA序列读取数目正确
基因编码长度合理
gtgaccgagctcgaccgggccttcgacgccgtaccggccccgatctacacccaccatgaacggcacggcgaaacggtgcaccgctcggcaccggagtcgatccgccgcgagctggccgccctgcaggtccgcgccggggaccgggtgctcgagatcggtacgggctccgggtacagcggcgcgctgctcgcgcacctgtgctgcccggacggccaggtcaccagcatcgacatcagcgacgaactcgtccgccgcgcggcagccatccacgccgagcgcggggtaaccagcgtcgactgccatgttggcgacggactggccggctacccggccgcagcgcccttccatcgagctgtttcgtggtgtgctccgccgcggctgccgagggcgtggacgcagcaggtcgtgaacggtgggcggatcgtcgcgtgcctgccgatcacggccctgccgtcgaccacgttgatcgccaccatcaccgttgcggccgggaaacctcgtatcgaagccctcgccggaggcggctacgcccagagcacgcccgtcgcggtcgacgatgccctgaccgtccccggccgctgggtcgactactgcgaccgccagcccgatccgtcctggatcggcatctgctggcgttctgccgacgacgcccagcacaccggcgctcgctccgccctcggccagctgctgcatcccgggtacaccgacacgtatcggcagatggagccgcactggcgctcctggtacacctggacgtccgcgctcggcgacccgcagctgagcctggtgtcgctgcgcaacgagatccgcggactcggtcacaccacaccaagttcggcggcggtgatcctgaccgatggcagggtcatcgcggaccggccggactcgccgtccctgcgctccttgcggacctggctgcaacggtgggaacacgtcggccgtccggcacccgagtccttcgcccgcaccctggtgccgc

In [98]:
# 单独处理join，以BGC0000003.gbk为例
reverse_dict = {'a':'t', 't':'a', 'g':'c', 'c':'g'}
file_name = 'BGC0000003.gbk'
file='mibig_gbk_3.1/'+'BGC0000003.gbk'
with open(file, 'r') as f:
    # 读取DNA序列
    text = f.readlines() # list格式
    text = ''.join(text) # string格式
    # 获取DNA序列
    DNA_len = int(re.search(r'[0-9]+ bp', text, flags = 0).group()[:-2])
    pattern = re.compile(r'ORIGIN[\s\S]+')
    DNA_seq_num = re.search(pattern, text, flags = 0).group()[7:]
    DNA_seq = re.sub(r'[0-9 \n/]*', '', DNA_seq_num)
    if(DNA_len != len(DNA_seq)):
        print("文件"+file+"出错：")
        print("DNA序列读取数目错误")
#         return

    # 获取基因区块起止
    gene_pattern = re.compile(r'gene            ')
    # gene的匹配需要后面的空格以指示每块的开头，基因互补链也是同样的开头所以可以识别
    gene_set_start = [substr.start() for substr in re.finditer(gene_pattern, text)] + [text.find('ORIGIN')]
    # 因为每个基因可以由多个编码序列（CDS）组成

    # 定义写入文件
    gene_filename = (file_name).replace('.gbk', '.gene')
    amino_acid_filename = (file_name).replace('.gbk', '.aa')
    output_dir = 'output'
    write_file_gene = open(output_dir + '/' + gene_filename, 'w')
    write_file_aa = open(output_dir + '/' + amino_acid_filename, 'w')
    count = 0
    # 获取CDS区块
    for i in range(len(gene_set_start)-1):
        # 基因区块
        gene_block = text[gene_set_start[i]:gene_set_start[i+1]]
        CDS_pattern = re.compile(r'CDS             ')
        # 识别CDS，同样可识别complement
        # CDS区块起止
        CDS_set_start = [substr.start() for substr in re.finditer(CDS_pattern, gene_block)] + [gene_set_start[i+1]] 

        # 存在多个CDS的情况
        for j in range(int(len(CDS_set_start))-1):
            count += 1
            # CDS区块
            CDS_block = gene_block[CDS_set_start[j]:CDS_set_start[j+1]]
            # 识别起始需要特别注意，包括complement 和 join等两种类别
            # 识别最基础正义链的起始
            begin2end_pattern_normal = re.compile(r'CDS             ([0-9]+)..([0-9]+)')
            # 识别不常规的情况需要使用字符串搜索
#             begin2end_pattern_abnormal = re.compile(r'CDS             [.+]\)\n') # 识别不常规的情况

            begin2end_str = re.search(begin2end_pattern_normal, CDS_block, flags=0)
            if not begin2end_str:
                begin2end_str = CDS_block[:CDS_block.find(')\n')]
                print(begin2end_str)
                begin2end_pattern_abnormal = re.compile(r'[0-9]+')
                begin2end_num = re.findall(begin2end_pattern_abnormal, begin2end_str)
                if 'complement' in begin2end_str:
                    reverse = True
                len_begin2end_num = len(begin2end_num)
                if len_begin2end_num%2 != 0:
                    print("起止数目不为偶数，出现错误！")
                len_begin2end_num //= 2
                gene_seq = ''
                for num in range(len_begin2end_num):
                    startN = int(begin2end_num[num*2])-1
                    endN = int(begin2end_num[num*2+1])
                    gene_seq += DNA_seq[startN:endN]
                if 'complement' in begin2end_str:
                    gene_seq = ''.join([reverse_dict[ch] for ch in gene_seq])[::-1]
            else:
                startN = int(begin2end_str.group(1))-1 # 索引从0开始
                endN = int(begin2end_str.group(2)) # 包含最后字符
                gene_seq = DNA_seq[startN:endN]
            translation_pattren = re.compile(r'/translation="([A-Z\s]+)"')
            translation = re.search(translation_pattren, CDS_block, flags=0).group(1)
            translation = translation.replace(' ', '').replace('\n', '')
            if len(gene_seq)%3 != 0:
                print("基因编码长度不合理")
#                 return
            # 文件写入
            write_file_gene.write(file.split('/')[-1].replace('.gbk', '')+'_gene_'+str(count)+':')
            write_file_gene.write(gene_seq+'\n')
            write_file_aa.write(file.split('/')[-1].replace('.gbk', '')+'_protein_'+str(count)+':')
            write_file_aa.write(translation+'\n')
    print("已处理", count, "个基因")
    write_file_gene.close()
    write_file_aa.close()

#             print(begin2end_str.group(1), begin2end_str.group(2))
#     print(gene_set_start)
#     print(text[1951:4402])
#     print(text[4402:5319])


CDS             complement(join(12402..18796,18852..21474)
CDS             join(22504..22683,23066..23378,23434..23661,23717..23806,
                     23859..24346
CDS             complement(join(24799..25472,25522..25732,25786..26188,
                     26241..26299,26362..26544)
CDS             join(35716..35999,36059..36280,36351..36596,36652..36790
已处理 6 个基因


In [20]:
from Bio import SeqIO


In [43]:
record_iterator = SeqIO.parse('mibig_gbk_3.1/'+'BGC0000001.gbk', 'genbank')
record = list(record_iterator)
print(record)
# print(record[0].seq)
print(len(record))
print(dir(record[0]))
print(len(record[0]))
print(record[0].seq)

[SeqRecord(seq=Seq('GTGACCGAGCTCGACCGGGCCTTCGACGCCGTACCGGCCCCGATCTACACCCAC...TAA'), id='BGC0000001', name='BGC0000001', description='Verrucosispora maris AB-18-032 protein methyltransferase (orfP), putative hydrolase, pathway-specific SARP activator (abyR), cytochrome P450 (abyX), LuxR family transcriptional regulator (abyH), pathway-specific SARP activator (abyI), YD repeat protein (abyK), 3-oxoacyl-ACP synthase III (abyA1), phosphatase and glyceryl transferase (abyA2), acyl-carrier protein (abyA3), pyruvate/2-oxoglutarate dehydrogenase (abyA4), AbyA5 (abyA5), type 1 polyketide synthase (abyB1), type 1 polyketide synthase (abyB2), type 1 polyketide synthase (abyB3), TetR regulatory protein (abyC), EmrB/QacA drug resistance transporter (abyD), luciferase/monooxygenase (abyE), BC transporter oligopeptide binding protein (abyF1), ABC transporter oligopeptide permease (abyF2), binding-protein dependent ABC transporter (abyF3), ATP-binding ABC peptide transporter (abyF4), cytochrome P450 (

In [28]:
import os
import re
reverse_dict = {'a':'t', 't':'a', 'g':'c', 'c':'g', 'n':'n'}
def process_gbk(file):
    file_name = file.split('/')[-1]
    print("处理文件："+file)
    with open(file, 'r') as f:
    # 读取DNA序列
        text = f.readlines() # list格式
        text = ''.join(text) # string格式
        # 获取DNA序列
        DNA_len = int(re.search(r'[0-9]+ bp', text, flags = 0).group()[:-2])
        pattern = re.compile(r'ORIGIN[\s\S]+')
        DNA_seq_num = re.search(pattern, text, flags = 0).group()[7:]
        DNA_seq = re.sub(r'[0-9 \n/]*', '', DNA_seq_num)
        if(DNA_len != len(DNA_seq)):
            print("文件"+file+"出错：")
            print("DNA序列读取数目错误")
            return

#         # 获取基因区块起止
#         gene_pattern = re.compile(r'gene            ')
#         # gene的匹配需要后面的空格以指示每块的开头，基因互补链也是同样的开头所以可以识别
#         gene_set_start = [substr.start() for substr in re.finditer(gene_pattern, text)] + [text.find('ORIGIN')]
#         # 因为每个基因可以由多个编码序列（CDS）组成
#         if len(gene_set_start)<=1:
#             gene_set_start = [1] + gene_set_start
#         # 定义写入文件
        gene_filename = (file_name).replace('.gbk', '.gene')
        amino_acid_filename = (file_name).replace('.gbk', '.aa')
        output_dir = 'output'
        write_file_gene = open(output_dir + '/' + gene_filename, 'w')
        write_file_aa = open(output_dir + '/' + amino_acid_filename, 'w')
        count = 0
        
        CDS_pattern = re.compile(r'     CDS             ')
        CDS_set_start = [substr.start() for substr in re.finditer(CDS_pattern, text)] + [text.find('ORIGIN')]
        for i in range(len(CDS_set_start)-1):
            count += 1
            CDS_block = text[CDS_set_start[i]:CDS_set_start[i+1]]
            begin2end_str = CDS_block[:CDS_block.find('\n')]
            if 'join' in begin2end_str:
                begin2end_str = CDS_block[:CDS_block.find(')\n')]
            begin2end_pattern = re.compile(r'[0-9]+')
            begin2end_num = re.findall(begin2end_pattern, begin2end_str)
            len_begin2end_num = len(begin2end_num)
            if len_begin2end_num%2 != 0:
                print("起止数目不为偶数，出现错误！")
                continue
#                 return
            len_begin2end_num //= 2
            gene_seq = ''
            for num in range(len_begin2end_num):
                startN = int(begin2end_num[num*2])-1
                endN = int(begin2end_num[num*2+1])
                gene_seq += DNA_seq[startN:endN]
            if 'complement' in begin2end_str:
                gene_seq = ''.join([reverse_dict[ch] for ch in gene_seq])[::-1]
            translation_pattren = re.compile(r'/translation="([A-Z\s]+)"')
            translation = re.search(translation_pattren, CDS_block, flags=0).group(1)
            translation = translation.replace(' ', '').replace('\n', '')
            
#             print(gene_seq)
#             print(begin2end_num)
#             print(begin2end_str)
#             print(len(gene_seq))
#             print(translation)
            
            if len(gene_seq)%3 != 0 and (not '<' in CDS_block) and (not '>' in CDS_block):
                print("基因编码长度不合理", len(gene_seq))
                print(gene_seq)
                print(begin2end_num)
                print(begin2end_str)
                print(len(gene_seq))
                print(translation)
#                 return
            if abs(len(gene_seq)//3 - (len(translation)-1))>=3:
                print("基因长度与蛋白质长度不对应")
                print("基因长度/3:", len(gene_seq)//3, "\n蛋白质长度：", len(translation)-1)
#                 return
            # 文件写入
            write_file_gene.write(file.split('/')[-1].replace('.gbk', '')+'_gene_'+str(count)+':')
            write_file_gene.write(gene_seq+'\n')
            write_file_aa.write(file.split('/')[-1].replace('.gbk', '')+'_protein_'+str(count)+':')
            write_file_aa.write(translation+'\n')            

#         # 获取CDS区块
#         for i in range(len(gene_set_start)-1):
#             # 基因区块
#             gene_block = text[gene_set_start[i]:gene_set_start[i+1]]
#             CDS_pattern = re.compile(r'CDS             ')
#             # 识别CDS，同样可识别complement
#             # CDS区块起止
#             CDS_set_start = [substr.start() for substr in re.finditer(CDS_pattern, gene_block)] + [gene_set_start[i+1]] 

#             # 存在多个CDS的情况
#             for j in range(int(len(CDS_set_start))-1):
#                 count += 1
#                 # CDS区块
#                 CDS_block = gene_block[CDS_set_start[j]:CDS_set_start[j+1]]
#                 # 识别起始需要特别注意，包括complement 和 join等两种类别
#                 # 识别最基础正义链的起始
#                 begin2end_pattern_normal = re.compile(r'CDS             ([0-9]+)..([0-9]+)')
#                 # 识别不常规的情况需要使用字符串搜索
#     #             begin2end_pattern_abnormal = re.compile(r'CDS             [.+]\)\n') # 识别不常规的情况

#                 begin2end_str = re.search(begin2end_pattern_normal, CDS_block, flags=0)
#                 if not begin2end_str:
#                     begin2end_str = CDS_block[:CDS_block.find(')\n')]
#                     print(begin2end_str)
#                     begin2end_pattern_abnormal = re.compile(r'[0-9]+')
#                     begin2end_num = re.findall(begin2end_pattern_abnormal, begin2end_str)
#                     if 'complement' in begin2end_str:
#                         reverse = True
#                     len_begin2end_num = len(begin2end_num)
#                     if len_begin2end_num%2 != 0:
#                         print("起止数目不为偶数，出现错误！")
#                         return
#                     len_begin2end_num //= 2
#                     gene_seq = ''
#                     for num in range(len_begin2end_num):
#                         startN = int(begin2end_num[num*2])-1
#                         endN = int(begin2end_num[num*2+1])
#                         gene_seq += DNA_seq[startN:endN]
#                     if 'complement' in begin2end_str:
#                         gene_seq = ''.join([reverse_dict[ch] for ch in gene_seq])[::-1]
#                 else:
#                     startN = int(begin2end_str.group(1))-1 # 索引从0开始
#                     endN = int(begin2end_str.group(2)) # 包含最后字符
#                     gene_seq = DNA_seq[startN:endN]
#                 translation_pattren = re.compile(r'/translation="([A-Z\s]+)"')
#                 translation = re.search(translation_pattren, CDS_block, flags=0).group(1)
#                 translation = translation.replace(' ', '').replace('\n', '')
#                 if len(gene_seq)%3 != 0:
#                     print("基因编码长度不合理")
#                     return
#                 # 文件写入
#                 write_file_gene.write(file.split('/')[-1].replace('.gbk', '')+'_gene_'+str(count)+':')
#                 write_file_gene.write(gene_seq+'\n')
#                 write_file_aa.write(file.split('/')[-1].replace('.gbk', '')+'_protein_'+str(count)+':')
#                 write_file_aa.write(translation+'\n')
                
                

        print("已处理", count, "个基因")
        write_file_gene.close()
        write_file_aa.close()
            
files = os.listdir('mibig_gbk_3.1')
# print(files)
ii = 0            
for file in files:
    ii += 1
    file_path = 'mibig_gbk_3.1' + '/' + file
    process_gbk(file_path)
#     if ii == 14:
#         break

    
    
#     with open(file, 'r') as f:
#         text = f.readlines() # list格式
#         text = ''.join(text) # string格式
#         # 获取DNA序列
#         DNA_len = int(re.search(r'[0-9]+ bp', text, flags = 0).group()[:-2])
#         pattern = re.compile(r'ORIGIN[\s\S]+')
#         DNA_seq_num = re.search(pattern, text, flags = 0).group()[7:]
#         DNA_seq = re.sub(r'[0-9 \n/]*', '', DNA_seq_num)
#         if(DNA_len == len(DNA_seq)):
#             print("DNA序列读取数目正确")
#         else:
#             print("DNA序列读取数目错误")
#     #         exit(1)
#         gene_pattern = re.compile(r'gene            ')
#         # gene的匹配需要后面的空格以指示每块的开头，基因互补链也是同样的开头所以可以识别
#         gene_set_start = [substr.start() for substr in re.finditer(gene_pattern, text)] + [text.find('ORIGIN')]
#         # 因为每个基因可以由多个编码序列（CDS）组成
#         for i in range(len(gene_set_start)-1):
#             gene_block = text[gene_set_start[i]:gene_set_start[i+1]]
#             CDS_pattern = re.compile(r'CDS             ')
#             # 识别CDS，同样可识别complement
#             CDS_set_start = [substr.start() for substr in re.finditer(CDS_pattern, gene_block)] + [gene_set_start[i+1]] 
#     #         print(CDS_set_start)
#     #         print(gene_block)
#             # 存在多个CDS的情况
#             for j in range(int(len(CDS_set_start))-1):
#                 CDS_block = gene_block[CDS_set_start[j]:CDS_set_start[j+1]]
#                 # 识别正义链的起始
#                 begin2end_pattern_1 = re.compile(r'CDS             ([0-9]+)..([0-9]+)')
#                 # 识别反义链的起始
#                 begin2end_pattern_2 = re.compile(r'CDS             complement\(([0-9]+)..([0-9]+)\)')
#                 reverse = False
#                 begin2end_str = re.search(begin2end_pattern_1, CDS_block, flags=0)
#                 if not begin2end_str:
#                     begin2end_str = re.search(begin2end_pattern_2, CDS_block, flags=0)
#                     reverse = True
#                 translation_pattren = re.compile(r'/translation="([A-Z\s]+)"')
#                 translation = re.search(translation_pattren, CDS_block, flags=0).group(1)
#                 translation = translation.replace(' ', '').replace('\n', '')
#                 startN = int(begin2end_str.group(1))-1 # 索引从0开始
#                 endN = int(begin2end_str.group(2)) # 包含最后字符
#                 gene_seq = DNA_seq[startN:endN]
#                 if reverse:
#                     gene_seq = ''.join([reverse_dict[ch] for ch in gene_seq])
#                 if len(gene_seq)%3==0:
#                     print("基因编码长度合理")
#                 else:
#                     print("基因编码长度不合理")
#     #                 break
#                 print(gene_seq)
#                 print(translation)
#                 print(CDS_block)
#                 print(begin2end_str)
#     #             print(begin2end_str.group(1), begin2end_str.group(2))
#     #     print(gene_set_start)
#     #     print(text[1951:4402])
#     #     print(text[4402:5319])

处理文件：mibig_gbk_3.1/BGC0000001.gbk
已处理 28 个基因
处理文件：mibig_gbk_3.1/BGC0000002.gbk
已处理 34 个基因
处理文件：mibig_gbk_3.1/BGC0000003.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000004.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0000006.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0000007.gbk
已处理 25 个基因
处理文件：mibig_gbk_3.1/BGC0000008.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0000009.gbk
已处理 25 个基因
处理文件：mibig_gbk_3.1/BGC0000010.gbk
已处理 22 个基因
处理文件：mibig_gbk_3.1/BGC0000011.gbk
已处理 26 个基因
处理文件：mibig_gbk_3.1/BGC0000012.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0000013.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0000014.gbk
已处理 34 个基因
处理文件：mibig_gbk_3.1/BGC0000016.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000017.gbk
已处理 15 个基因
处理文件：mibig_gbk_3.1/BGC0000018.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0000019.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0000020.gbk
已处理 41 个基因
处理文件：mibig_gbk_3.1/BGC0000021.gbk
已处理 39 个基因
处理文件：mibig_gbk_3.1/BGC0000022.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0000023.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0000024.gbk
已处理 9 个基因
处理文件：mibig_gbk_3

已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000197.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0000198.gbk
基因编码长度不合理 287
ctggagaccttccgtgtccccgggggagcgttcacccgcgtggacacccatgcctaccccggctaccgggtcccgcccggctacgactcgctgctggcgaagctgatcacctgggcgccgcggcgggagcaggccctagagcggatggaccgggcccttgccggttcgcgatcgccggtccgggagtgcggaccaccgttccgttcctccggcggatcctgtgtgagccgtcgttccggtgcggcgagcacacgacgtccctcatcgccggcgtcctggctcgatag
['23130', '23416']
     CDS             complement(23130..23416)
287
LETFRVPGGAFTRVDTHAYPGYRVPPGYDSLLAKLITWAPRREQALERMDRALAGSRSPVRECGPPFRSSGGSCVSRRSGAASTRRPSSPASWLD
基因长度与蛋白质长度不对应
基因长度/3: 124 
蛋白质长度： 86
已处理 42 个基因
处理文件：mibig_gbk_3.1/BGC0000199.gbk
已处理 37 个基因
处理文件：mibig_gbk_3.1/BGC0000200.gbk
已处理 35 个基因
处理文件：mibig_gbk_3.1/BGC0000201.gbk
已处理 29 个基因
处理文件：mibig_gbk_3.1/BGC0000202.gbk
已处理 45 个基因
处理文件：mibig_gbk_3.1/BGC0000203.gbk
已处理 42 个基因
处理文件：mibig_gbk_3.1/BGC0000204.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0000205.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000207.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0000208.gbk
已处理

已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0000348.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000349.gbk
已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0000350.gbk
已处理 25 个基因
处理文件：mibig_gbk_3.1/BGC0000351.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000352.gbk
已处理 17 个基因
处理文件：mibig_gbk_3.1/BGC0000353.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000354.gbk
已处理 33 个基因
处理文件：mibig_gbk_3.1/BGC0000355.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0000356.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0000357.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0000359.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0000360.gbk
已处理 28 个基因
处理文件：mibig_gbk_3.1/BGC0000362.gbk
已处理 39 个基因
处理文件：mibig_gbk_3.1/BGC0000363.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0000364.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0000365.gbk
已处理 30 个基因
处理文件：mibig_gbk_3.1/BGC0000366.gbk
已处理 18 个基因
处理文件：mibig_gbk_3.1/BGC0000367.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0000368.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0000369.gbk
已处理 32 个基因
处理文件：mibig_gbk_3.1/BGC0000371.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0000372.gbk
已处理 8 个基因
处理文件：mib

已处理 24 个基因
处理文件：mibig_gbk_3.1/BGC0000465.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0000466.gbk
已处理 31 个基因
处理文件：mibig_gbk_3.1/BGC0000467.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0000468.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0000469.gbk
已处理 33 个基因
处理文件：mibig_gbk_3.1/BGC0000470.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0000471.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000472.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0000473.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0000474.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0000475.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000476.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0000477.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000478.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0000479.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0000480.gbk
已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0000481.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0000482.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0000483.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000484.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000485.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000486.gbk
已处理 5 个基因
处理文件：mibig_gbk

已处理 44 个基因
处理文件：mibig_gbk_3.1/BGC0000610.gbk
已处理 26 个基因
处理文件：mibig_gbk_3.1/BGC0000611.gbk
已处理 27 个基因
处理文件：mibig_gbk_3.1/BGC0000612.gbk
已处理 21 个基因
处理文件：mibig_gbk_3.1/BGC0000613.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0000614.gbk
已处理 21 个基因
处理文件：mibig_gbk_3.1/BGC0000615.gbk
已处理 24 个基因
处理文件：mibig_gbk_3.1/BGC0000616.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000617.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0000618.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000619.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0000620.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0000621.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0000622.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0000623.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0000624.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0000625.gbk
已处理 15 个基因
处理文件：mibig_gbk_3.1/BGC0000626.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000627.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0000628.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0000630.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0000631.gbk
已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0000632.gbk
已处理 13 个基因
处理文件：mibi

...

已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0002270.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002271.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002272.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0002273.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0002274.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0002275.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0002276.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0002277.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0002278.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0002282.gbk
已处理 28 个基因
处理文件：mibig_gbk_3.1/BGC0002283.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002284.gbk
已处理 15 个基因
处理文件：mibig_gbk_3.1/BGC0002285.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002286.gbk
已处理 1 个基因
处理文件：mibig_gbk_3.1/BGC0002287.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0002288.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0002289.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0002290.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0002291.gbk
已处理 24 个基因
处理文件：mibig_gbk_3.1/BGC0002293.gbk
已处理 16 个基因
处理文件：mibig_gbk_3.1/BGC0002294.gbk
已处理 26 个基因
处理文件：mibig_gbk_3.1/BGC0002295.gbk
已处理 5 个基因
处理文件：mibig

已处理 34 个基因
处理文件：mibig_gbk_3.1/BGC0002385.gbk
已处理 21 个基因
处理文件：mibig_gbk_3.1/BGC0002386.gbk
已处理 20 个基因
处理文件：mibig_gbk_3.1/BGC0002387.gbk
已处理 34 个基因
处理文件：mibig_gbk_3.1/BGC0002389.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0002390.gbk
已处理 19 个基因
处理文件：mibig_gbk_3.1/BGC0002391.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0002392.gbk
已处理 16 个基因
处理文件：mibig_gbk_3.1/BGC0002393.gbk
已处理 17 个基因
处理文件：mibig_gbk_3.1/BGC0002394.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0002395.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002396.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0002397.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0002398.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0002399.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002400.gbk
已处理 3 个基因
处理文件：mibig_gbk_3.1/BGC0002401.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0002402.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0002403.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002404.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002405.gbk
已处理 4 个基因
处理文件：mibig_gbk_3.1/BGC0002406.gbk
已处理 20 个基因
处理文件：mibig_gbk_3.1/BGC0002407.gbk
已处理 7 个基因
处理文件：mibig_gb

已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0002496.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002497.gbk
已处理 44 个基因
处理文件：mibig_gbk_3.1/BGC0002498.gbk
已处理 49 个基因
处理文件：mibig_gbk_3.1/BGC0002499.gbk
已处理 24 个基因
处理文件：mibig_gbk_3.1/BGC0002500.gbk
已处理 16 个基因
处理文件：mibig_gbk_3.1/BGC0002501.gbk
已处理 26 个基因
处理文件：mibig_gbk_3.1/BGC0002502.gbk
基因长度与蛋白质长度不对应
基因长度/3: 411 
蛋白质长度： 130
已处理 16 个基因
处理文件：mibig_gbk_3.1/BGC0002503.gbk
已处理 12 个基因
处理文件：mibig_gbk_3.1/BGC0002504.gbk
已处理 22 个基因
处理文件：mibig_gbk_3.1/BGC0002505.gbk
已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0002506.gbk
已处理 23 个基因
处理文件：mibig_gbk_3.1/BGC0002507.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0002508.gbk
已处理 11 个基因
处理文件：mibig_gbk_3.1/BGC0002509.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0002510.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002512.gbk
已处理 6 个基因
处理文件：mibig_gbk_3.1/BGC0002514.gbk
已处理 15 个基因
处理文件：mibig_gbk_3.1/BGC0002515.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0002517.gbk
已处理 25 个基因
处理文件：mibig_gbk_3.1/BGC0002518.gbk
已处理 2 个基因
处理文件：mibig_gbk_3.1/BGC0002520.gbk
已处理 59 个基因
处理文件：mibig_g

基因长度与蛋白质长度不对应
基因长度/3: 206 
蛋白质长度： 118
已处理 25 个基因
处理文件：mibig_gbk_3.1/BGC0002668.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0002669.gbk
基因长度与蛋白质长度不对应
基因长度/3: 84 
蛋白质长度： 47
已处理 42 个基因
处理文件：mibig_gbk_3.1/BGC0002670.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0002671.gbk
已处理 33 个基因
处理文件：mibig_gbk_3.1/BGC0002672.gbk
已处理 29 个基因
处理文件：mibig_gbk_3.1/BGC0002673.gbk
已处理 22 个基因
处理文件：mibig_gbk_3.1/BGC0002674.gbk
已处理 5 个基因
处理文件：mibig_gbk_3.1/BGC0002675.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0002676.gbk
已处理 76 个基因
处理文件：mibig_gbk_3.1/BGC0002677.gbk
已处理 18 个基因
处理文件：mibig_gbk_3.1/BGC0002678.gbk
已处理 14 个基因
处理文件：mibig_gbk_3.1/BGC0002679.gbk
已处理 10 个基因
处理文件：mibig_gbk_3.1/BGC0002680.gbk
已处理 13 个基因
处理文件：mibig_gbk_3.1/BGC0002681.gbk
已处理 7 个基因
处理文件：mibig_gbk_3.1/BGC0002682.gbk
已处理 8 个基因
处理文件：mibig_gbk_3.1/BGC0002683.gbk
已处理 10 个基因
处理文件：mibig_gbk_3.1/BGC0002685.gbk
已处理 9 个基因
处理文件：mibig_gbk_3.1/BGC0002686.gbk
基因编码长度不合理 1727
atgagacacccctccgcgcgcggccgccctgccgccgcgcgccgcggcctggcggccgcgtcggcggccgtcctgctcgcctcgggcacggtcgcgctggcccag

已处理 4 个基因


In [102]:
os.listdir('output')

['BGC0000001.aa',
 'BGC0000001.gene',
 'BGC0000002.aa',
 'BGC0000002.gene',
 'BGC0000003.aa',
 'BGC0000003.gene',
 'BGC0000004.aa',
 'BGC0000004.gene',
 'BGC0000006.aa',
 'BGC0000006.gene',
 'BGC0000007.aa',
 'BGC0000007.gene',
 'BGC0000008.aa',
 'BGC0000008.gene',
 'BGC0000009.aa',
 'BGC0000009.gene',
 'BGC0000010.aa',
 'BGC0000010.gene',
 'BGC0000011.aa',
 'BGC0000011.gene',
 'BGC0000012.aa',
 'BGC0000012.gene',
 'BGC0000013.aa',
 'BGC0000013.gene',
 'BGC0000014.aa',
 'BGC0000014.gene',
 'BGC0000016.aa',
 'BGC0000016.gene',
 'BGC0000017.aa',
 'BGC0000017.gene',
 'BGC0000018.aa',
 'BGC0000018.gene',
 'BGC0000019.aa',
 'BGC0000019.gene',
 'BGC0000020.aa',
 'BGC0000020.gene',
 'BGC0000021.aa',
 'BGC0000021.gene',
 'BGC0000022.aa',
 'BGC0000022.gene',
 'BGC0000023.aa',
 'BGC0000023.gene',
 'BGC0000024.aa',
 'BGC0000024.gene',
 'BGC0000025.aa',
 'BGC0000025.gene',
 'BGC0000026.aa',
 'BGC0000026.gene',
 'BGC0000027.aa',
 'BGC0000027.gene',
 'BGC0000028.aa',
 'BGC0000028.gene',
 'BGC0000029

In [107]:
aa = ''
for aafile in os.listdir('output'):
    if '.aa' in aafile:
        with open('output/'+aafile, 'r') as f:
            text = f.readlines()
#             print(text)
        for line in text:
            aa += '>'+line.replace(':', '\n')
with open('output/mibig.fa', 'w') as f:
    f.write(aa)

In [110]:
with open('output/mibig.fa', 'r') as f:
    text = f.readlines()
    halfLenOfText = len(text)//2
    text1 = text[:halfLenOfText]
    text2 = text[halfLenOfText:]
    with open('output/mibig1.fa', 'w') as f:
        f.write(''.join(text1))
    with open('output/mibig2.fa', 'w') as f:
        f.write(''.join(text2))
        

In [111]:
with open('output/mibig1.fa', 'r') as f:
    text = f.readlines()
    print(len(text))

30361


In [1]:
with open('output/mibig1.fa', 'r') as f:
    text = f.readlines()
    halfLenOfText = len(text)//2
    text1 = text[:halfLenOfText]
#     text2 = text[halfLenOfText:]
    with open('output/mibig1.5.fa', 'w') as f:
        f.write(''.join(text1))
#     with open('output/mibig2.fa', 'w') as f:
#         f.write(''.join(text2))
        

In [2]:
with open('output/mibig1.5.fa', 'r') as f:
    text = f.readlines()
    print(len(text))

15180
